# Explore the Data

In [ ]:
import pandas as pd
df = pd.read_csv('nonvoters_data.csv')

df.head()

: 

In [ ]:
df.info()

By looking at the info from our dataframe, we learn that we have 119 columns in our dataset and 5,836 rows. 

There's too many columns to see the full number and so, to find out information about the columns, [we refer to the codebook for the dataset](https://github.com/fivethirtyeight/data/blob/master/non-voters/nonvoters_codebook.pdf).

This analysis will focus on questions two and three in the study and so we'll filter the dataframe to only include the pertinent columns.


2. In your view, how important are each of the following to being a good American?

1. Voting in elections
2. Serving on a jury
3. Following what happens in government and politics
4. Displaying the American flag
5. Participating in the U.S. Census every ten years
6. Knowing the Pledge of Allegiance
7. Supporting the military
8. Respecting the opinions of those who disagree with you
9. Believing in God
10. Protesting if you believe government actions are wrong
    
Responses
1. Very important
2. Somewhat important
3. Not so important
4. Not at all important


3. How much do you agree or disagree with the following statements?

1. Systemic racism is a problem in the United States.
2. Systemic racism in policing is a bigger problem than violence and vandalism in protests.
3. Society as a whole has become too soft and feminine.
4. The mainstream media is more interested in making money than telling the truth.
5. Traditional parties and politicians don’t care about people like me.
6. The way people talk needs to change with the times to be more sensitive to people with different backgrounds.

Responses
1. Strongly agree
2. Somewhat agree
3. Somewhat disagree
4. Strongly disagree

In [ ]:
df.describe()

Now that we've inspected our data, we can make a list of the ways we'll need to restructure the data.

1) Create bins for our age data: create quartiles for age data
2) Filter dataframe for the specific questions: questions two and three 
3) Remove values of -1 
4) Likert Scale data needs to be relabeled
5) Rename Columns

# Clean and Restructure the Data

## Create bins for our age data: create quartiles for age data

In [ ]:
df["ppage"].describe()

# Use quartile values to create bins

age_bins = ["22-36","37-54","55-65","66-94"]

df['age_binned'] = pd.qcut(df['ppage'],
                              q=[0,0.25, 0.5, 0.75, 1],
                              labels=age_bins)

df


## Filter dataframe for the specific questions

In [ ]:
import re

# Create new dataframe that only includes demographic and Questions two and three. 
# We'll use these lists to create our final dataframe

full_column_list = list(df.columns)

question_columns = []

demographic_columns = []

for column in full_column_list:
    # Append any columns that contain the text Q2_ or Q2_
    if re.search("Q2_.+", column) or re.search("Q3_.+", column):
        question_columns.append(column)
    # Append demographic criteria to list of list of demographic variables
    elif df[column].dtype == object:
        demographic_columns.append(column)
        # TO DO: Remove and replace with binned data
    elif re.search("age_binned",column):
        demographic_columns.append(column)

# Create full list of columns we'll use for our subset
full_column_list = (question_columns+demographic_columns)

# Create a copy of the dataframe we'll use for our final df
subset_df = df[full_column_list].copy()


In [ ]:
# Remove values that are below zero 

for name in question_columns:
    subset_df.drop(subset_df[subset_df[name]<0].index, inplace = True)

subset_df.describe()

# Use Lambda function to relabel Likert scale data



In [ ]:
def likert_converter (column_name):
    """Replace Likert Scale numeric values with words"""
    if 'Q2_' in column_name:
        subset_df[column_name] = subset_df[column_name].map(lambda x: 'Very Important' if x==1 
        else ('Somewhat important' if x==2 
        else('Not so important'if x==3 
        else 'Not at all important')))
    else:
        subset_df[column_name] = subset_df[column_name].map(lambda x: 'Very Important' if x==1 
        else ('Somewhat important' 
        if x==2 else('Not so important'if x==3 
        else 'Not at all important')))


#Replace with correct column list
for column in question_columns: 
    likert_converter(column)

 


## Rename columns


In [ ]:
column_dict = {"Q2_1":"Voting",
        "Q2_2": "Jury",
        "Q2_3": "Govt_Politics",
        "Q2_4": "American_Flag",
        "Q2_5": "US_Census",
        "Q2_6": "Pledge",
        "Q2_7":"Military Support",
        "Q2_8": "Respecting_Others",
        "Q2_9":"Believing in God",
        "Q2_10":"Protesting", 
        "Q3_1": "Racism_US",
        "Q3_2": "Racism_Policing",
        "Q3_3":"Society_Whole",
        "Q3_4":"Mainstream_Media",
        "Q3_5":"Traditional_Parties",
        "Q3_5":"Way_People_Talk"}


subset_df = subset_df.rename(columns=column_dict)



In [ ]:
# Checking for missing values for our dataframe
subset_df.isna().sum().sort_values()

## Export df into csv file

In [ ]:
subset_df.to_csv('american_values.csv')

# Perform exploratory analysis & visualization using Matplotlib and Seaborn

In [ ]:
# Create visualizations for dataset 

# Ask and Answer questions about the data in a Jupyter Notebook

- Do people of certain backgrounds tend to think differently American values?
- Do certain answers correlate with each other?

# Summarize your inference and write a conclusion
